In [1]:
import ee, os
import geemap
ee.Initialize()
Map = geemap.Map()
# Map = geemap.Map(center=(39, 110), zoom=4)
# Map

In [2]:
year = 2019, 2021
day = '-01-01', '-12-31'

start_date = str(year[0]) + day[0]
end_date = str(year[1]) + day[1]

# jx_bound = ee.FeatureCollection('users/yehuigeo/jx_bound')
fw = geemap.shp_to_ee(r'D:\work\new\shp\fw.shp').geometry()
# Map.addLayer(fw,{},'fw')

In [3]:
def fun(image):
    return (image.select('tmmx').add(image.select('tmmn'))).multiply(0.5).rename('tem')

def NDVI(img):
    ndvi = img.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    return img.addBands(ndvi)

def EVI(img):
    nir = img.select("SR_B4")
    red = img.select("SR_B3")
    blue = img.select("SR_B1")
    evi = img.expression("2.5*(B4 - B3)/(B4 + 6*B3 - 7.5*B1 + 1)",{"B4": nir,"B3": red ,"B1": blue})
    evi = evi.rename('EVI')
    return img.addBands(evi)

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);

#   // Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    
    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBands, None, True)\
                 .updateMask(qaMask)\
                 .updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

def K(image):
    return image.expression(
        'pr/aet',{
            'pr':image.select('pr'),
            'aet':image.select('aet').multiply(0.1)
        }
    ).rename('K')

def sample_points_to_df(img, ps, col_names=None):
    ps_data = img.sampleRegions(ps, 
                                geometries=True, 
                                scale=30.0)
    values_list = [f['properties'] for f in ps_data.getInfo()["features"]]
    if col_names == None:
        values_df = pd.DataFrame(values_list)
    else:
        values_df = pd.DataFrame(values_list)[col_names+['CID']]
    return values_df

def pro(img):
    return img.resample('bicubic').reproject(**{'crs': 'EPSG:4362', 'scale': 30.0})

In [4]:
# CRU  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
            .filterBounds(fw) \
            .filterDate(start_date, end_date) \
            .filterMetadata('CLOUD_COVER', 'less_than', 25)\
            .map(maskL8sr) \
            .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])\
            .map(NDVI).map(EVI)

dataset = (ee.Image('USGS/SRTMGL1_003').clip(fw))
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation).rename('slope')

images3 = ee.Image([elevation,slope])
# print(collection.size().getInfo())

In [5]:
image1 = []
image2 = []

for i in range(1,13):
#     print('{}_'.format(i))
    image = collection.filter(ee.Filter.calendarRange(i, i, 'month'))\
                .sort('CLOUD_COVER', False)\
                .mosaic().clip(fw)
    image1.append(image.select(['NDVI','EVI']))
    
    precipitation = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(fw)\
                .filter(ee.Filter.calendarRange(i, i, 'month'))\
                .select('pr')\
                .sum().clip(fw)
    image2.append(precipitation)
    
    temperature = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(fw)\
                .filter(ee.Filter.calendarRange(i, i, 'month'))\
                .select(['tmmn','tmmx'])\
                .map(fun).mean().multiply(0.1)\
                .clip(fw)
    image2.append(temperature)
    
    Kdata = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(fw)\
                .filter(ee.Filter.calendarRange(i, i, 'month'))\
                .select(['pr','aet'])
    Ks = Kdata.map(K).mean().float().clip(fw)
    image2.append(Ks)

images1 = ee.Image(image1)
images2 = ee.Image(image2)

# image5 = collection.filter(ee.Filter.calendarRange(5, 5, 'month'))\
#                 .sort('CLOUD_COVER', False)\
#                 .mosaic()

# image9 = collection.filter(ee.Filter.calendarRange(9, 9, 'month'))\
#                 .sort('CLOUD_COVER', False)\
#                 .mosaic()

In [6]:
name1 = []
name2 = []
name3 = ['elevation','slope']

for i in range(1,13):
    name1.append('NDVI_{}'.format(i))
    name1.append('EVI_{}'.format(i))
    name2.append('pr_{}'.format(i))
    name2.append('tem_{}'.format(i))
    name2.append('K_{}'.format(i))

images1 = images1.rename(name1)
images2 = images2.rename(name2)

In [7]:
name = 6
shp = geemap.shp_to_ee(r'D:\work\new\point\6.shp')

In [8]:
import glob
import pandas as pd

path = r'D:\work\new\point'
outpath1 = r'D:\work\new\txt\NDVI'
outpath2 = r'D:\work\new\txt\tem'
outpath3 = r'D:\work\new\txt\dem'

# for name in range(1,8):
    
outfile1 = outpath1+os.sep+'{}.csv'.format(name)
outfile2 = outpath2+os.sep+'{}.csv'.format(name)
outfile3 = outpath3+os.sep+'{}.csv'.format(name)

#     if os.path.exists(outfile1) and os.path.exists(outfile2) and os.path.exists(outfile3):
#         continue

print('{}_'.format(name))

shp = geemap.shp_to_ee(path+os.sep+'{}.shp'.format(name)) # 这一步会很慢

df1 = sample_points_to_df(images1, shp, name1)
df2 = sample_points_to_df(images2, shp, name2)
df3 = sample_points_to_df(images3, shp, name3)

df1.to_csv(outfile1,index=False, header=True)
df2.to_csv(outfile2,index=False, header=True)
df3.to_csv(outfile3,index=False, header=True)

6_


In [9]:
# Map.addLayer(image5,{},"image5")
# Map.addLayer(image9,{},"image9")
# Map